In [84]:
import os
import sys
import numpy as np
from tqdm import tqdm
from PIL import Image
from font_diffuser.args import SampleArgs
from font_diffuser.sample import load_fontdiffuer_pipeline, sampling

class ImageConcat:
    def __init__(self, n_h, n_v, resolution=96):
        self.n_h = n_h
        self.n_v = n_v
        self.r = resolution
        self.img = Image.new("RGB", (self.n_h*self.r, self.n_v*self.r))
        self.cursor = [0,0]
        
    def move_cursor_h(self):
        self.cursor[0] += self.r
        
    def move_cursor_v(self):
        self.cursor[1] += self.r
        
    def reset_cursor_h(self):
        self.cursor[0] = 0
        
    def append_img(self, im):
        self.img.paste(im, tuple(self.cursor))
        
    def save_img(self, path):
        self.img.save(path)

        
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

        
def black2blue(img):
    array_img = np.array(img)
    for i in range(96):
        for j in range(96):
            if not np.all(array_img[i,j,:] > 200):
                array_img[i,j,:] = (0,0,140)
    return Image.fromarray(array_img)

def black2gray(img):
    array_img = np.array(out_image, dtype=np.float32)
    array_img = array_img - 255
    array_img = array_img * 0.75
    array_img = array_img + 255
    return Image.fromarray(array_img.astype(np.uint8))

In [85]:
args = SampleArgs()
r = 96
n_h = 6
n_v = 12
contents = np.arange(0,11172,11172/(n_h*n_v), dtype=int)

In [ ]:
pipe = load_fontdiffuer_pipeline(args=args,model_i="p2_20000")
pbar = tqdm([p.split(".ttf")[0] for p in os.listdir("data/r40202/ttfs") if ("UhBee" in p) or ("twice" in p)])
for style in pbar:
    style_image_paths = [p for p in os.listdir("data/r40202/pngs/") if style in p]
    if ("p1_28__p2_2__%s.png"%style not in os.listdir("data/f40227/"))&(len(style_image_paths)>0):
        with HiddenPrints():
            bigimg = ImageConcat(n_h=n_h*2, n_v=n_v, resolution=r)
            style_image = Image.open("data/r40202/pngs/"+style_image_paths[0]).convert('RGB')
            c = 0
            for _ in range(n_v):
                for _ in range(n_h):
                    ci = contents[c]
                    content_image = Image.open("data/r40202/pngs/gulim__%s.png"%str(ci)).convert('RGB')
                    try:
                        bigimg.append_img(Image.open("data/r40202/pngs/%s__%s.png"%(str(style), str(ci))).convert('RGB'))
                    except:
                        bigimg.append_img(Image.open("data/f40219/noimg.png").convert('RGB'))
                    bigimg.move_cursor_h()

                    out_image = sampling(
                            args=args, 
                            pipe=pipe, 
                            content_image=content_image,
                            style_image=style_image,
                        )
                    bigimg.append_img(black2gray(out_image))
                    bigimg.move_cursor_h()

                    c += 1
                bigimg.reset_cursor_h()
                bigimg.move_cursor_v()
            bigimg.save_img("data/f40227/p1_28__p2_2__%s.png"%style)
            pbar.set_postfix(style=style)

Load the down block  DownBlock2D
Load the down block  MCADownBlock2D
The style_attention cross attention dim in Down Block 1 layer is 1024
The style_attention cross attention dim in Down Block 2 layer is 1024
Load the down block  MCADownBlock2D
The style_attention cross attention dim in Down Block 1 layer is 1024
The style_attention cross attention dim in Down Block 2 layer is 1024
Load the down block  DownBlock2D
Load the up block  UpBlock2D
Load the up block  StyleRSIUpBlock2D
Load the up block  StyleRSIUpBlock2D
Load the up block  UpBlock2D
Param count for Ds initialized parameters: 20591296
Get CG-GAN Style Encoder!
Param count for Ds initialized parameters: 1187008
Get CG-GAN Content Encoder!
Loaded the model state_dict successfully!
Loaded training DDPM scheduler sucessfully!
Loaded dpm_solver pipeline sucessfully!


 18%|█▊        | 31/170 [1:50:34<7:06:22, 184.05s/it, style=UhBee UJong]           